In [4]:
import pandas as pd
import numpy as np

# import the flat file we preprocessed after download from https://www.baseball-reference.com/leagues/majors/
chi_square_df = pd.read_csv('mahler_test_data.csv').set_index('Year')

# quick sense check of df
chi_square_df.head()

,ATL,BAL,BOS,CHC,CHW,CIN,CLE,DET,LAD,MIN,NYY,OAK,PHI,PIT,SFG,STL
Year,,,,,,,,,,,,,,,,
2023,0.380000,0.410000,0.490000,0.550000,0.590000,0.480000,0.530000,0.560000,0.450000,0.500000,0.500000,0.730000,0.480000,0.580000,0.460000,0.560000
2022,0.376543,0.487654,0.518519,0.543210,0.500000,0.617284,0.432099,0.592593,0.314815,0.518519,0.388889,0.629630,0.462963,0.617284,0.500000,0.425926
2021,0.456790,0.679012,0.432099,0.561728,0.425926,0.487654,0.506173,0.524691,0.345679,0.549383,0.432099,0.469136,0.493827,0.623457,0.339506,0.444444
2020,0.416667,0.583333,0.600000,0.433333,0.416667,0.483333,0.416667,0.616667,0.283333,0.400000,0.450000,0.400000,0.533333,0.683333,0.516667,0.500000
2019,0.401235,0.666667,0.481481,0.481481,0.555556,0.537037,0.425926,0.709877,0.345679,0.376543,0.364198,0.401235,0.500000,0.574074,0.524691,0.438272


In [9]:
# create numpy array of teams
array_of_teams = np.array([np.array(chi_square_df[col]) for col in chi_square_df])

# get means of each array
array_of_team_means = np.mean(array_of_teams, axis = 1)

# assume 150 games per year to get estimate of actual losses (this is exactly what Mahler does in the paper)
actual_losses = array_of_teams*150

# calculate expected losses for each team as grand mean of each column * 150
expected_losses = 150*array_of_team_means

# put expected losses into array of same shape as actual losses
expected_losses_array = np.tile(expected_losses, (121,1)).transpose()

# generate the chi square statistics for each team
chi_sqs = np.sum(((actual_losses - expected_losses_array)**2)/expected_losses_array, axis =1)

print(chi_sqs)

[299.57233647 275.04158666 239.86343761 236.25637036 185.20674365
 189.4131277  178.72917378 211.72163758 233.09768319 227.4543068
 233.38021503 398.4638349  242.25858216 249.10668461 188.33563365
 230.53525334]


In [13]:
from scipy.stats import chi2

# the degrees of freedom should be 1 less than the number of categories, which in our example is simply the number of years 
dof = 120

# use chi2 from scipy to calculate the p value, which is just a shortcut for looking up the test statistics on a table
p_values = [1 - chi2.cdf(i, dof) for i in chi_sqs]

p_values

[0.0,
 3.441691376337985e-14,
 5.212387188535672e-10,
 1.3126194575718841e-09,
 0.00012378044437011404,
 5.496387028747218e-05,
 0.00040904274765996007,
 4.852030366375715e-07,
 2.9154270109188474e-09,
 1.1820436962572956e-08,
 2.7157170956471077e-09,
 0.0,
 2.8030244791921177e-10,
 4.6159409627932746e-11,
 6.784504732437746e-05,
 5.527827195983548e-09]

In [12]:
from scipy.stats import chisquare

# as a sense check and future shortcut, just use chisquare from scipy to get the p values directly
results = [chisquare(i, j) for (i, j) in zip(actual_losses, expected_losses_array)]
results

[Power_divergenceResult(statistic=299.5723364686352, pvalue=2.3657042883350046e-17),
 Power_divergenceResult(statistic=275.041586655234, pvalue=3.440373542271784e-14),
 Power_divergenceResult(statistic=239.86343760726368, pvalue=5.212387279713363e-10),
 Power_divergenceResult(statistic=236.2563703635706, pvalue=1.3126194394830656e-09),
 Power_divergenceResult(statistic=185.2067436546074, pvalue=0.0001237804443700934),
 Power_divergenceResult(statistic=189.4131276965877, pvalue=5.4963870287457885e-05),
 Power_divergenceResult(statistic=178.7291737832756, pvalue=0.00040904274765993085),
 Power_divergenceResult(statistic=211.72163757562623, pvalue=4.852030366064987e-07),
 Power_divergenceResult(statistic=233.09768319128744, pvalue=2.915427058583078e-09),
 Power_divergenceResult(statistic=227.45430680007152, pvalue=1.1820436932483048e-08),
 Power_divergenceResult(statistic=233.38021502925105, pvalue=2.71571704509783e-09),
 Power_divergenceResult(statistic=398.4638348993134, pvalue=1.399801